In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastbook import *
from fastai.imports import *
from fastai.tabular.all import *
import os

In [3]:
path = Path(os.path.expandvars("${HOME}/.fastai/data/titanic"))
path.ls()

(#4) [Path('/home/jmd/.fastai/data/titanic/gender_submission.csv'),Path('/home/jmd/.fastai/data/titanic/titanic.zip'),Path('/home/jmd/.fastai/data/titanic/train.csv'),Path('/home/jmd/.fastai/data/titanic/test.csv')]

In [4]:
df = pd.read_csv(f"{path}/train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Explore Pandas

In [28]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [30]:
df['Parch'].unique()

array([0, 1, 2, 5, 3, 4, 6])

In [51]:
df.describe(include='all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"van Melkebeke, Mr. Philemon",male,NaN,NaN,NaN,CA. 2343,NaN,B96 B98,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [17]:
df.iloc[:,0] # Show all ids

0        1
1        2
2        3
3        4
4        5
      ... 
886    887
887    888
888    889
889    890
890    891
Name: PassengerId, Length: 891, dtype: int64

In [18]:
df.iloc[0,:]

PassengerId                          1
Survived                             0
Pclass                               3
Name           Braund, Mr. Owen Harris
Sex                               male
Age                                 22
SibSp                                1
Parch                                0
Ticket                       A/5 21171
Fare                              7.25
Cabin                              NaN
Embarked                             S
Name: 0, dtype: object

In [19]:
df.iloc[0]

PassengerId                          1
Survived                             0
Pclass                               3
Name           Braund, Mr. Owen Harris
Sex                               male
Age                                 22
SibSp                                1
Parch                                0
Ticket                       A/5 21171
Fare                              7.25
Cabin                              NaN
Embarked                             S
Name: 0, dtype: object

## Data Setup

## Metric
The metric is the percentage of passengers you correctly predict (i.e accuracy).

In [5]:
def create_vars():
    cat_names  = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
    cont_names = ['PassengerId', 'Age', 'Fare', 'SibSp', 'Parch', 'Pclass']
    procs = [Categorify, FillMissing, Normalize]
    return cat_names, cont_names, procs

In [6]:
cat_names, cont_names, procs = create_vars()

In [7]:
dls = TabularDataLoaders.from_df(
    df, path, procs=procs, cat_names=cat_names, cont_names=cont_names,
    y_names="Survived", y_block=CategoryBlock, bs=64)

In [8]:
dls.show_batch()

,Name,Sex,Ticket,Cabin,Embarked,Age_na,PassengerId,Age,Fare,SibSp,Parch,Pclass,Survived
0,"McGowan, Miss. Anna ""Annie""",female,330923,#na#,Q,False,22.999998,15.000000,8.029199,1.689237e-09,-9.897945e-09,3.0,1
1,"Kelly, Mr. James",male,363592,#na#,S,False,696.999993,44.000000,8.050001,1.689237e-09,-9.897945e-09,3.0,0
2,"Van der hoef, Mr. Wyckoff",male,111240,B19,S,False,171.000013,60.999999,33.500000,1.689237e-09,-9.897945e-09,1.0,0
3,"Sage, Mr. Douglas Bullen",male,CA. 2343,#na#,S,True,847.000013,28.000000,69.550003,8.000000e+00,2.000000e+00,3.0,0
4,"Danoff, Mr. Yoto",male,349219,#na#,S,False,321.999997,27.000000,7.895800,1.689237e-09,-9.897945e-09,3.0,0
5,"Daly, Mr. Peter Denis",male,113055,E17,S,False,858.000014,51.000001,26.549999,1.689237e-09,-9.897945e-09,1.0,1
6,"Garside, Miss. Ethel",female,243880,#na#,S,False,576.999992,34.000000,13.000001,1.689237e-09,-9.897945e-09,2.0,1
7,"Gustafsson, Mr. Alfred Ossian",male,7534,#na#,S,False,877.000001,20.000000,9.845801,1.689237e-09,-9.897945e-09,3.0,0
8,"Crosby, Capt. Edward Gifford",male,WE/P 5735,B22,S,False,745.999985,70.000000,71.000001,1.000000e+00,1.000000e+00,1.0,0
9,"Bourke, Mrs. John (Catherine)",female,364849,#na#,Q,False,657.999992,32.000000,15.500000,1.000000e+00,1.000000e+00,3.0,0


## Model Setup

In [9]:
learn = tabular_learner(dls, metrics=accuracy)

In [10]:
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.683865,0.680625,0.601124,00:00
1,0.530002,0.645467,0.612360,00:00
2,0.364020,0.610481,0.646067,00:00
3,0.253549,0.572051,0.713483,00:00
4,0.184769,0.530246,0.786517,00:00


## Data Setup with Tabular Pandas

In [11]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df)) # Split into train, valid

In [12]:
cat_names, cont_names, procs = create_vars()
to = TabularPandas(
    df, procs=procs, cat_names=cat_names, cont_names=cont_names,
    y_names='Survived', y_block = CategoryBlock, splits=splits)

In [13]:
to.xs.iloc[:2]

,Name,Sex,Ticket,Cabin,Embarked,Age_na,PassengerId,Age,Fare,SibSp,Parch,Pclass
491,872,2,650,0,3,1,0.173943,-0.606557,-0.492462,-0.478385,-0.474181,0.812356
567,628,1,396,0,3,1,0.470672,0.013127,-0.222015,-0.478385,4.425685,0.812356


In [14]:
dls = to.dataloaders(bs=64)

In [15]:
dls.show_batch()

,Name,Sex,Ticket,Cabin,Embarked,Age_na,PassengerId,Age,Fare,SibSp,Parch,Pclass,Survived
0,"Dennis, Mr. Samuel",male,A/5 21172,#na#,S,False,320.999999,22.000000,7.250000,-2.187958e-09,1.730021e-09,3.0,0
1,"Watt, Mrs. James (Elizabeth ""Bessie"" Inglis Milne)",female,C.A. 33595,#na#,S,False,162.000008,40.000000,15.750000,-2.187958e-09,1.730021e-09,2.0,1
2,"Birkeland, Mr. Hans Martin Monsen",male,312992,#na#,S,False,409.000001,21.000000,7.775000,-2.187958e-09,1.730021e-09,3.0,0
3,"Isham, Miss. Ann Elizabeth",female,PC 17595,C49,C,False,178.000012,49.999999,28.712499,-2.187958e-09,1.730021e-09,1.0,0
4,"Mellors, Mr. William John",male,SW/PP 751,#na#,S,False,226.999995,19.000000,10.500000,-2.187958e-09,1.730021e-09,2.0,1
5,"Hickman, Mr. Lewis",male,S.O.C. 14879,#na#,S,False,666.000007,32.000000,73.499999,2.000000e+00,1.730021e-09,2.0,0
6,"Stanley, Miss. Amy Zillah Elsie",female,CA. 2314,#na#,S,False,650.000003,23.000000,7.550001,-2.187958e-09,1.730021e-09,3.0,1
7,"Dodge, Master. Washington",male,33638,A34,S,False,446.000000,4.000000,81.858299,-2.187958e-09,2.000000e+00,1.0,1
8,"Lobb, Mrs. William Arthur (Cordelia K Stanlick)",female,A/5. 3336,#na#,S,False,617.999996,26.000000,16.100000,1.000000e+00,1.730021e-09,3.0,0
9,"Banfield, Mr. Frederick James",male,C.A./SOTON 34068,#na#,S,False,884.000002,28.000000,10.500000,-2.187958e-09,1.730021e-09,2.0,0


In [16]:
learn = tabular_learner(dls, metrics=accuracy)

In [17]:
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.660477,0.671747,0.595506,00:00
1,0.502047,0.651178,0.601124,00:00
2,0.339108,0.633789,0.601124,00:00
3,0.234542,0.600704,0.623595,00:00
4,0.170374,0.549482,0.702247,00:00


In [19]:
learn.show_results()

,Pclass,Name,Sex,Parch,Ticket,Cabin,Embarked,Age_na,PassengerId,Age,Fare,SibSp,Survived,Survived_pred
0,2.0,857.0,2.0,1.0,550.0,0.0,3.0,1.0,-1.608345,2.842242,-0.443387,-0.463553,0.0,1.0
1,3.0,418.0,1.0,1.0,492.0,0.0,3.0,1.0,-1.299495,-0.710254,-0.457390,0.408514,0.0,1.0
2,3.0,65.0,1.0,2.0,212.0,0.0,1.0,1.0,-0.338198,1.220450,-0.361355,-0.463553,0.0,1.0
3,2.0,749.0,1.0,1.0,166.0,0.0,3.0,1.0,1.148145,0.062027,-0.391523,-0.463553,1.0,0.0
4,3.0,414.0,1.0,3.0,676.0,0.0,3.0,2.0,1.692494,-0.092429,-0.174732,0.408514,0.0,0.0
5,3.0,150.0,2.0,1.0,298.0,0.0,3.0,1.0,-1.392150,-0.401342,-0.494213,-0.463553,0.0,1.0
6,2.0,345.0,1.0,2.0,572.0,0.0,3.0,1.0,-0.037069,1.220450,-0.116645,0.408514,1.0,1.0
7,1.0,370.0,1.0,1.0,72.0,93.0,3.0,1.0,1.217636,1.683819,0.956072,0.408514,1.0,1.0
8,3.0,503.0,2.0,1.0,540.0,0.0,3.0,2.0,0.055587,-0.092429,-0.494213,-0.463553,0.0,1.0


In [38]:
row, clas, probs = learn.predict(df.iloc[0])
row.show()

,Name,Sex,Ticket,Cabin,Embarked,Age_na,PassengerId,Age,Fare,SibSp,Parch,Pclass,Survived
0,"Braund, Mr. Owen Harris",male,A/5 21171,#na#,S,False,0.999987,22.0,7.249999,1.0,1.741594e-09,3.0,0


In [22]:
clas, probs

(tensor(1), tensor([0.4866, 0.5134]))

In [23]:
test_df = df.copy()
test_df.drop(['Survived'], axis=1, inplace=True)
dl = learn.dls.test_dl(test_df)

In [24]:
learn.get_preds(dl=dl)

(tensor([[0.4866, 0.5134],
         [0.4937, 0.5063],
         [0.4887, 0.5113],
         ...,
         [0.5061, 0.4939],
         [0.4994, 0.5006],
         [0.5023, 0.4977]]),
 None)

## Set up Data for Test Set

In [ ]:
test_df = pd.read_csv(f"{path}/test.csv")
test_df.head()